In [3]:
import pandas as pd
import time, os,requests,shutil
import concurrent.futures
pd.options.mode.chained_assignment = None
columns = shutil.get_terminal_size().columns

In [4]:
def connection_check():
    API = '0a66e9aecdc8c49191309691d8c340b920a060b3'
    endpoint = 'https://cmdb.netledger.com/api/dc-hosts/'
    headers = {'Authorization': 'Token '+ API}
    response = requests.get(endpoint, headers=headers)
    return response

def ip_search (ip):
    API = '0a66e9aecdc8c49191309691d8c340b920a060b3'
    ip = ip
    endpoint = 'https://cmdb.netledger.com/api/ipaddresses/?address={ip}'
    url = endpoint.format(ip=ip)
    headers = {'Authorization': 'Token '+ API}
    response = requests.get(url, headers=headers)
    result = response.json()
    count = result['count']
    if count == 1:
        print(str(ip) + " IP IN CMDB ")
        data['IP in CMDB'][data['IP'] == ip] = 'IP in CMDB'
    elif count < 1:
        print(str(ip) + " IP NOT in CMDB ")
        data['IP in CMDB'][data['IP'] == ip] = 'IP NOT in CMDB'
    elif count > 1:
        print(str(ip) + " IP IN CMDB + ")
        data['IP in CMDB'][data['IP'] == ip] = 'IP in CMDB+'

def outdated(dns):
    API = '0a66e9aecdc8c49191309691d8c340b920a060b3'
    url = f'https://cmdb.netledger.com/api/dc-hosts/?hostname=Outdated%20({dns})'
    headers = {'Authorization': 'Token '+ API}
    response = requests.get(url, headers=headers)
    result = response.json()
    count = result['count']
    if count == 1:
            data['DNS in CMDB'][data['DNS'] == dns] = 'DNS in CMDB - Outdated'
            print(dns,'DNS in CMDB - Outdated')
    elif count < 1:
            data['DNS in CMDB'][data['DNS'] == dns] = 'DNS NOT in CMDB'
            print(str(dns)," DNS NOT in cmdb")  
    

def dns_search (dns):
    API = '0a66e9aecdc8c49191309691d8c340b920a060b3'
    url = f'https://cmdb.netledger.com/api/dc-hosts/?hostname={dns}'
    headers = {'Authorization': 'Token '+ API}
    response = requests.get(url, headers=headers)
    result = response.json()
    count = result['count']
    if count == 0:
        count = outdated(dns)
    elif count == 1:
        data['DNS in CMDB'][data['DNS'] == dns] = 'DNS in CMDB'
        print(str(dns)," DNS in cmdb")
    elif count > 1:
        data['DNS in CMDB'][data['DNS'] == dns] = 'DNS in CMDB + '
        
def vmhost_search (dns):
    API = '0a66e9aecdc8c49191309691d8c340b920a060b3'
    dns = dns
    endpoint = 'https://cmdb.netledger.com/api/virtual-servers/?hostname={dns}'
    url = endpoint.format(dns=dns)
    headers = {'Authorization': 'Token '+ API}
    response = requests.get(url, headers=headers)
    result = response.json()
    count = result['count']
    if count != 0:
        url = result['results'][0]['hypervisor']['hostname']
        data['VM host URL'][data['DNS'] == dns] = url
        print(url)
    elif count == 0 :
        data['VM host URL'][data['DNS'] == dns] = 'blank'
        print('blank')
    else:
        url = dns
        data['VM host URL'][data['DNS'] == dns] = url
        print(url) 
        
def vm_check(dns):
    if '.f.' in dns:
        vmhost_search(dns)
    elif '.m.' in dns:
        vmhost_search(dns)
    elif '.snap.' in dns:
        vmhost_search(dns)
    else:
        pass

In [5]:
print(r"""
   _____ __  __ _____  ____     ____                        
  / ____|  \/  |  __ \|  _ \   / __ \                       
 | |    | \  / | |  | | |_) | | |  | |_   _  ___ _ __ _   _ 
 | |    | |\/| | |  | |  _ <  | |  | | | | |/ _ \ '__| | | |
 | |____| |  | | |__| | |_) | | |__| | |_| |  __/ |  | |_| |
  \_____|_|  |_|_____/|____/   \___\_\\__,_|\___|_|   \__, |
                                                       __/ |
                                                      |___/ 
""")


   _____ __  __ _____  ____     ____                        
  / ____|  \/  |  __ \|  _ \   / __ \                       
 | |    | \  / | |  | | |_) | | |  | |_   _  ___ _ __ _   _ 
 | |    | |\/| | |  | |  _ <  | |  | | | | |/ _ \ '__| | | |
 | |____| |  | | |__| | |_) | | |__| | |_| |  __/ |  | |_| |
  \_____|_|  |_|_____/|____/   \___\_\\__,_|\___|_|   \__, |
                                                       __/ |
                                                      |___/ 



In [6]:

print("".center(columns,'*'))
print("This script is for querying CMDB using an existing raw merged data workbook".center(columns))
print("It will check if the the data already have results in CMDB first before it queries the rest or just the empty ones".center(columns))
print("You will need your CMDB API Token".center(columns))
print("It may take ~30 mins for the script to finish depending on the size of the data needed to be queried".center(columns))
print("Make sure that you're connected to the VPN".center(columns))
print("For questions regarding the script you can contact Raenielle at rsunga@netsuite.com".center(columns))
print("".center(columns,'*'))


*****************************************************************************************************************
                   This script is for querying CMDB using an existing raw merged data workbook                   
It will check if the the data already have results in CMDB first before it queries the rest or just the empty ones
                                        You will need your CMDB API Token                                        
       It may take ~30 mins for the script to finish depending on the size of the data needed to be queried      
                                    Make sure that you're connected to the VPN                                   
               For questions regarding the script you can contact Raenielle at rsunga@netsuite.com               
*****************************************************************************************************************


In [7]:
API = input("Enter you CMDB API ")
print("Checking VPN  Connection...")

if connection_check().status_code == 200:
    print("VPN Connection GOOD")
else:
    while connection_check().status_code != 200:
        checking = input("Please check VPN Connection and then press 'Enter' to continue\n")
        connection_check().status_code == 200

Enter you CMDB API  0a66e9aecdc8c49191309691d8c340b920a060b3


Checking VPN  Connection...
VPN Connection GOOD


In [8]:
file = input("enter raw merged data file name location ex: ~/Downloads/merged_qualys_scan (no file extension): \n")
file_check = os.path.isfile(file+'.xlsx')
while file_check == False:
    file = input("enter raw merged data file name location ex: ~/Downloads/merged_qualys_scan (no file extension): \n")
    file_check = os.path.isfile(file+'.xlsx')
print("File found: " + str(file_check))
file =file+'.xlsx'

enter raw merged data file name location ex: ~/Downloads/merged_qualys_scan (no file extension): 
 CMDB_Queried_Data_Latest_Test


File found: True


In [9]:
start = time.perf_counter()
workbook = pd.ExcelFile(file)

In [10]:
data = pd.read_excel(workbook)
x = data[data['IP in CMDB'].isnull()].index
y = data[data['DNS in CMDB'].isnull()].index

In [11]:
while len(x) != 0 or len(y) !=0:
    if len(x) < 20 or len(y) < 20:
        choice = input("There only "+str(len(x))+" IPs and "+ str(len(y))+ " DNS that has not been run in CMDB. Would you like to run 'all' or 'selected' ?:  ").lower()
        while choice not in ['selected','all']:
            choice = input("There only "+str(len(x))+" IPs and "+str(len(y))+" DNS that has not been run in CMDB. Would you like to run 'all' or 'selected' ?:  ").lower()
        if choice == "all":
            with concurrent.futures.ThreadPoolExecutor() as executor:
                executor.map(ip_search, data['IP'])
                executor.map(dns_search, data['DNS'])
        elif choice == 'selected':
            for index in x:
                ip_search(data['IP'][data['IP'].index==index].to_string(index=False).lstrip())
            for index in y:
                dns_search(data['DNS'][data['DNS'].index==index].to_string(index=False).lstrip())
    else:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(ip_search, data['IP'])
            executor.map(dns_search, data['DNS'])
    
    x = data[data['IP in CMDB'].isnull()].index
    y = data[data['DNS in CMDB'].isnull()].index




There only 2 IPs and 1 DNS that has not been run in CMDB. Would you like to run 'all' or 'selected' ?:   selected


64.89.44.99 IP NOT in CMDB 
212.25.249.74 IP NOT in CMDB 
fs020.prod.sea.netledger.com  DNS in cmdb


In [13]:

    
with pd.ExcelWriter('CMDB_Queried_Data_Latest_Test.xlsx') as writer:
    data.to_excel(writer, index=False)
stop = time.perf_counter()

print("It took "+str(round((stop-start)/60,2))+" minutes for the script to complete")

It took 0.54 minutes for the script to complete
